In [1]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

1812

In [3]:
with open(files[0], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\00107855-d79b-4efb-808e-68c3a6816077.json'

In [5]:
json_data['data']['attributes']

{'id': '00107855-d79b-4efb-808e-68c3a6816077'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Competitive',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 1433161,
 'dateStarted': '2022-08-07T22:08:58.634+00:00',
 'rounds': 15,
 'isRanked': True,
 'queueId': 'competitive',
 'map': 'e2ad5c54-4114-a870-9641-8ea21279579a',
 'mapName': 'Icebox',
 'mapImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/maps/icebox.jpg'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T11:04:22.5577585+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## round_summary

In [11]:
len(round_summary)

15

In [12]:
# round_summary[22]


In [13]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Blue'

## player_summary

In [14]:
len(player_summary)

10

In [15]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [16]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'voidedrose#4822'}

In [17]:
player_summary[0]['metadata']

{'partyId': 'e853526d-a4ff-4e57-9a7f-23e9a8de3030',
 'teamId': 'Blue',
 'agentKey': 'a3bfb853-43b2-7238-a4f1-ad90e9e46bcc',
 'agentName': 'Reyna',
 'agentColor': '#f650d3',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/a3bfb853-43b2-7238-a4f1-ad90e9e46bcc/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/reyna_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'voidedrose#4822',
  'platformUserIdentifier': 'voidedrose#4822',
  'avatarUrl': None,
  'additionalParameters': None}}

In [18]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [19]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/6.png'},
 'value': 'Bronze 1',
 'displayValue': 'Bronze 1',
 'displayType': 'String'}

In [20]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 5262,
 'displayValue': '5,262',
 'displayType': 'Number'}

In [21]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 350.8,
 'displayValue': '351',
 'displayType': 'Number'}

## Win prediction data set creation

In [22]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']

In [23]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for i in roles_list:
            if i == agent_role:
                new_record[f'role_{i}'] = 1
            else:
                new_record[f'role_{i}'] = 0
        
        for i in all_agents_list:
            if i == agent:
                new_record[f'agent_{i}'] = 1
            else:
                new_record[f'agent_{i}'] = 0
            
        data.append(new_record)
            
    return data
        
        
processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary)
# processed_match_records[8]

In [24]:


def create_agent_row(agent_list, map_pick, game_win):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
        
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            
    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team))
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team))
    
    return [row1, row2]
    
    




In [25]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\40e4c768-3234-4769-b9db-06d71e5841e2.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\534b2ba1-8ff3-4ba9-995d-e3a5c5424bcf.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\630c9412-0424-4503-9565-dd1b98801516.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\63f2fd4b-3c5c-429d-9dd4-6706cd313e74.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\70cecd79-7a06-46dd-a9d8-3d579a645aa2.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\850a4d81-1be9-4a6c-b572-5242368fe21e.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\9bf7e777-b139-425e-801e-1dea4a219ca0.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\c0e973a1-fefc-4780-a07e-ab1fabefc150.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\dbcce03c-68eb-4633-9f23-5203f1e06c03.json
error C:\Users\trist\OneDrive\Documents\game_data\valor

(18060, 3604)

In [26]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'Thug Twerker#money', 'count': 27},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': 'strawberry milk#우유갠디', 'count': 25},
 {'user': 'Daddy Brim#OTP', 'count': 25},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'Arowana#queer', 'count': 23},
 {'user': 'DaddyMingze#NA1', 'count': 22},
 {'user': 'No Heads Hawes#2025', 'count': 22},
 {'user': 'hardstuckiron#3570', 'count': 21},
 {'user': 'DarkPegasis123#NA1', 'count': 21},
 {'user': '32Equr#6990', 'count': 21},
 {'user': 'Baguette#sus', 'count': 21},
 {'user': 'theramsaregood#00000', 'count': 21},
 {'user': 'suwucide#0210', 'count': 21},
 {'user': 'Ptownisawesome#5521', 'count': 21},
 {'user': 'Tengen#4835', 'count': 21},
 {'user': '04에르#000', 'count': 21}]

In [27]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

3306

In [28]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
1,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
2,DarkPegasis123#NA1,2022-08-06T17:49:51.751+00:00,0,Blue,Pearl,Gold 3,2898,144.900000,0.400000,8,...,0,0,0,0,0,0,0,0,0,1
3,Apples#1706,2022-08-04T21:23:53.819+00:00,0,Blue,Pearl,Bronze 2,3300,137.500000,0.416667,10,...,0,0,0,0,0,0,1,0,0,0
4,DOGLOVER32198367#WOOF,2022-08-04T21:23:53.819+00:00,0,Blue,Pearl,Iron 2,8897,370.708333,1.333333,32,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3301,fork#9925,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Bronze 1,2257,118.789474,0.421053,8,...,0,0,0,0,0,0,0,0,0,1
3302,Dark Dick#2499,2022-08-02T06:46:59.463+00:00,1,Blue,Ascent,Unranked,7557,397.736842,1.368421,26,...,0,0,0,0,1,0,0,0,0,0
3303,Ham#6386,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Bronze 1,5296,331.000000,1.125000,18,...,0,1,0,0,0,0,0,0,0,0
3304,Hitchman224#1988,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Unranked,3455,215.937500,0.750000,12,...,0,0,0,0,0,0,0,0,0,0


In [29]:
all_records_df['name'].nunique()

234

In [30]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [31]:
all_records_df['map_pick'].value_counts(normalize=True)

Fracture    0.151240
Ascent      0.146400
Haven       0.146098
Bind        0.139141
Breeze      0.138838
Icebox      0.137629
Pearl       0.131276
Split       0.009377
Name: map_pick, dtype: float64

In [79]:
all_records_df['killsPerRound'].describe()

count    3306.000000
mean        0.840226
std         1.630916
min         0.000000
25%         0.500000
50%         0.681818
75%         0.904762
max        39.000000
Name: killsPerRound, dtype: float64

In [33]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 1      0.140653
Bronze 2      0.133999
Bronze 3      0.115245
Silver 1      0.093769
Silver 3      0.083485
Iron 3        0.079552
Unranked      0.072898
Silver 2      0.061706
Iron 2        0.046884
Gold 1        0.041742
Gold 3        0.039625
Gold 2        0.035995
Platinum 1    0.025711
Platinum 2    0.013612
Platinum 3    0.007864
Iron 1        0.007260
Name: rank, dtype: float64

In [34]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [35]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

training_players, val_players

(['Skygamer1234#1234',
  'twink#7406',
  'Ben TenZ#baba',
  'Baguette#sus',
  'wakefull#icy',
  'skinnedbunnies#soup',
  'Mathematics#6622',
  'chocolate milk#MAKKE',
  'meamp26#8305',
  'Rotary#bwap',
  'Ham#6386',
  'YungCheddaBigD1k#2594',
  'notbottomfraggin#top',
  'tika#9504',
  'mogli35791#3087',
  'bottom frag#nodia',
  'WeePeaPie#4628',
  'judstar#NA1',
  'Winsswagz#fish',
  'QTRaju#4909',
  'Schleepers#Burns',
  'SonYujii#1846',
  'Tom#7622',
  'twistyyb#NA1',
  'strawberry milk#우유갠디',
  'girthysmurf#milfs',
  'KotaOfficial#6942',
  'TTV KABOWABO#2025',
  'mommys milk#slayy',
  'DOZ あ#54321',
  'NIX0#6770',
  'tenz jr#333',
  'FallenHelix07#0707',
  'CV WorstNub#2822',
  'sanrihoe#rawr',
  'Gojo#2011',
  'Swift Raptor#NA1',
  'thickdaddykrato#699',
  'DefNotASmurff#TOXIC',
  'FatCheekz0101#3953',
  'Big Sploinka#5371',
  'νегоiс#egirl',
  'ashe#918',
  'xBTGz#NA1',
  'kokushibo#3706',
  'Surrendeadthe2nd#jore',
  'ritz#4673',
  'Hugo#9108',
  'melon#goofy',
  'wind#3844',
  '

In [36]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [37]:
# training_data_df.isna().sum()

In [38]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5227686703096539

In [39]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.4,6110.6,1583.788022,4.938324,20.8,17.2,5.2,1.348095,3064.0,134.363733,6.4,2.2,2.0,0.0,0.0,2.0,13.5,16.0,6.5,1.5,0.467998,0.565321,0.229825,0.052949,0.6,0.6,1.6,1.4,40.333333,0.071128,0.062039,52.6,11.867244,74.933454,0.0,0.0,0.8,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0
1,0.4,7153.8,295.730879,1.045467,25.2,17.2,4.4,1.568095,4562.4,187.878018,8.8,3.2,2.6,0.6,0.0,3.2,14.8,18.2,5.8,1.2,0.610855,0.758178,0.251253,0.052949,0.6,0.6,2.6,1.4,140.333333,0.106843,0.062039,70.0,14.551808,71.362026,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0
2,0.4,7219.2,306.130879,1.090705,25.6,15.2,4.2,1.794935,4566.4,192.866114,9.2,3.6,2.4,0.6,0.0,3.0,14.2,19.0,2.8,0.8,0.601331,0.811749,0.132206,0.037473,0.4,0.4,2.8,1.0,173.666667,0.117557,0.045372,73.8,15.331028,77.790597,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.4,6633.2,285.301729,1.008924,23.4,14.0,3.4,1.709935,4234.8,182.448300,8.0,4.0,1.8,0.6,0.0,2.4,12.0,18.2,4.6,0.8,0.490400,0.818227,0.285242,0.042331,0.4,0.4,2.8,0.6,187.000000,0.122415,0.024319,69.4,15.956612,78.438370,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.2
4,0.2,7607.6,311.037721,1.112157,27.2,14.6,3.8,1.839697,4808.8,197.168558,7.8,4.6,2.2,1.0,0.0,3.2,14.8,19.8,5.6,1.2,0.566262,0.854003,0.316492,0.054185,0.2,0.2,3.4,0.4,192.142857,0.139225,0.016776,72.4,15.844856,82.145267,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,0.4,2994.0,165.599841,0.525998,9.6,14.0,2.4,0.735186,2029.0,111.872840,5.2,1.2,0.4,0.0,0.2,0.6,3.2,7.4,3.0,0.6,0.179733,0.405957,0.165815,0.030635,1.0,1.0,2.2,4.2,37.853480,0.116767,0.220735,50.8,19.986561,53.055138,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.8,0.0,0.0,0.0,0.0,0.0
12,0.4,3496.2,202.264841,0.628498,11.0,13.2,2.6,0.845361,2334.4,134.887840,4.6,2.2,0.4,0.0,0.2,0.6,2.8,8.2,4.0,0.6,0.164733,0.468457,0.233315,0.033135,1.0,1.0,2.2,2.8,43.238095,0.124267,0.158235,56.8,17.605608,57.8

In [40]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
28,past_5_games_avg_esr,0.030670
23,past_5_games_avg_ultimateCastsPerRound,0.031640
22,past_5_games_avg_ability2CastsPerRound,0.032123
29,past_5_games_avg_firstKillsPerRound,0.032687
30,past_5_games_avg_firstDeathsPerRound,0.033911
21,past_5_games_avg_ability1CastsPerRound,0.034079
31,past_5_games_avg_econRating,0.034636
20,past_5_games_avg_grenadeCastsPerRound,0.034704
33,past_5_games_avg_kast,0.036623
32,past_5_games_avg_hsAccuracy,0.037659


In [41]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5428051001821493

In [42]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

for i in training_data_df.columns:
    df_interaction[i] =  all_data_df[i]
    for j in training_data_df.columns:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_won_game,past_5_games_avg_score_mul_past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_s

In [43]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

,column,slope,intercept,rvalue,r2,pvalue,stderr
380,past_5_games_avg_assists_mul_past_5_games_avg_econRating,25.953297,266.458626,0.106213,0.011281,8.659879e-07,5.259622
1805,past_5_games_avg_econRating_mul_past_5_games_avg_assists,25.953297,266.458626,0.106213,0.011281,8.659879e-07,5.259622
413,past_5_games_avg_kdRatio_mul_past_5_games_avg_assists,0.553700,5.069711,0.093624,0.008765,1.464376e-05,0.127462
356,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.553700,5.069711,0.093624,0.008765,1.464376e-05,0.127462
529,past_5_games_avg_damagePerRound_mul_past_5_games_avg_assists,52.149056,626.247510,0.093474,0.008737,1.511220e-05,12.024043
358,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,52.149056,626.247510,0.093474,0.008737,1.511220e-05,12.024043
1798,past_5_games_avg_econRating,3.133399,56.947378,0.085941,0.007386,6.979685e-05,0.786333
471,past_5_games_avg_damage_mul_past_5_games_avg_assists,1045.590588,12772.222966,0.084442,0.007130,9.332408e-05,267.087777
357,past_5_games_avg_assists_mul_past_5_games_avg_damage,1045.590588,12772.222966,0.084442,0.007130,9.332408e-05,267.087777
438,past_5_games_avg_kdRatio_mul_past_5_games_avg_econRating,11.025766,67.963518,0.082820,0.006859,1.270899e-04,2.871979


In [44]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
31,past_5_games_avg_econRating,3.133399,56.947378,0.085941,0.007386,0.000070,0.786333
2,past_5_games_avg_scorePerRound,-70.592849,280.916400,-0.077304,0.005976,0.000349,19.708867
3,past_5_games_avg_killsPerRound,-0.220859,0.945535,-0.076292,0.005821,0.000417,0.062484
7,past_5_games_avg_kdRatio,0.080533,1.085876,0.075079,0.005637,0.000515,0.023154
9,past_5_games_avg_damagePerRound,5.246866,132.735969,0.071506,0.005113,0.000943,1.584345
8,past_5_games_avg_damage,104.713849,2670.716637,0.065390,0.004276,0.002498,34.591330
28,past_5_games_avg_esr,5.156073,59.751129,0.059774,0.003573,0.005720,1.863929
6,past_5_games_avg_assists,0.191200,4.730741,0.058757,0.003452,0.006601,0.070320
11,past_5_games_avg_doubleKills,0.116374,2.487703,0.058121,0.003378,0.007212,0.043270
5,past_5_games_avg_deaths,-0.260449,14.625497,-0.053768,0.002891,0.012942,0.104706



## Agent pick prediction data set creation

In [45]:
all_records, all_agent_records = get_all_processed_data()


error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\40e4c768-3234-4769-b9db-06d71e5841e2.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\534b2ba1-8ff3-4ba9-995d-e3a5c5424bcf.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\630c9412-0424-4503-9565-dd1b98801516.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\63f2fd4b-3c5c-429d-9dd4-6706cd313e74.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\70cecd79-7a06-46dd-a9d8-3d579a645aa2.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\850a4d81-1be9-4a6c-b572-5242368fe21e.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\9bf7e777-b139-425e-801e-1dea4a219ca0.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\c0e973a1-fefc-4780-a07e-ab1fabefc150.json
error C:\Users\trist\OneDrive\Documents\game_data\valorant_raw\dbcce03c-68eb-4633-9f23-5203f1e06c03.json
error C:\Users\trist\OneDrive\Documents\game_data\valor

In [46]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [47]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [48]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel,game_win
0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,2,0,0,0,2,2,4,0
1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,3,0,0,0,0,0,6,1
2,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,2,2,0,2,2,0,0,4,1
3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,1,3,1,0,0,0,3,1,3,0
4,0,0,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,2,0,0,0,0,0,6,1


In [49]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel
2991,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,2,1,1,2,1,1,2,2,1
2659,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,2,2,0,2,2,0,0,4
608,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,3,1,0,3,1,0,0,3
3178,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,1,1,2,1,1,2,1,2,2
2597,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,3,0,1,3,0,3,0,0


In [50]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

0.5066371681415929

In [51]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
14,agent_Sage,0.033470
22,map_Fracture,0.033535
24,map_Icebox,0.033578
23,map_Haven,0.033587
10,agent_Omen,0.033986
3,agent_Chamber,0.037380
6,agent_Jett,0.039759
11,agent_Phoenix,0.041380
12,agent_Raze,0.042342
13,agent_Reyna,0.047131


In [52]:
rf = LogisticRegression()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5165929203539823

In [53]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
9,agent_Neon,-0.046961,0.222465,-0.058779,3.454966e-03,0.002205,0.015326
5,agent_Fade,0.046076,0.177972,0.057511,3.307509e-03,0.002745,0.015370
36,role_Duelist*role_Sentinel,0.138162,2.187454,0.052364,2.742017e-03,0.006400,0.050633
30,role_Sentinel,0.071389,1.268417,0.051751,2.678135e-03,0.007048,0.026473
13,agent_Reyna,0.044928,0.567469,0.045666,2.085339e-03,0.017436,0.018887
35,role_Initiator*role_Sentinel,0.077998,0.982495,0.043146,1.861578e-03,0.024698,0.034707
6,agent_Jett,0.036555,0.342086,0.038072,1.449458e-03,0.047509,0.018437
18,agent_Yoru,-0.026018,0.192560,-0.033880,1.147881e-03,0.077829,0.014749
3,agent_Chamber,0.031056,0.424508,0.031281,9.785128e-04,0.103509,0.019069
1,agent_Breach,0.015881,0.070751,0.029504,8.704630e-04,0.124657,0.010339


In [54]:
from sklearn.decomposition import PCA

rf = LogisticRegression()
pca = PCA()

x_pca = pca.fit_transform(train_x)
x_val_pca = pca.transform(val_x)

rf.fit(x_pca, train_y)

preds = rf.predict(x_val_pca)

accuracy_score(preds, val_y)

0.5176991150442478

In [55]:
model= RandomForestClassifier(n_estimators=100, max_features = .9, max_samples=.5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.5099557522123894

In [56]:
# model.predict(val_x)

In [57]:
# model.predict_proba(val_x)[0][-1]

In [58]:
model= RandomForestClassifier(n_estimators=1000)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.5022123893805309

In [59]:
model= RandomForestClassifier(n_estimators=1000, max_features = .9, max_samples=.5)


train_df, val_df = train_test_split(all_agent_records_df)

all_x = all_agent_records_df.drop('game_win', axis = 1)
all_y = all_agent_records_df['game_win']

model.fit(all_x, all_y)


RandomForestClassifier(max_features=0.9, max_samples=0.5, n_estimators=1000)

In [60]:
def predict_best_lineup(model, map_pick, current_agent_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df = features_df.drop('game_win', axis = 1)
    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index
    preds = model.predict_proba(features_df)

    inputs_df['win_prob'] = model.predict_proba(features_df)[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = ['Phoenix',
                     'Reyna',
                      'Raze',
                     ]

predict_best_lineup(model, map_pick, current_agent_list)

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
4,Omen,Phoenix,Raze,Reyna,Skye,0.774833
16,Omen,Phoenix,Raze,Reyna,Sova,0.734417
58,Cypher,Fade,Phoenix,Raze,Reyna,0.714583
65,Jett,Omen,Phoenix,Raze,Reyna,0.709167
88,Jett,Phoenix,Raze,Reyna,Sage,0.692167
60,Fade,Phoenix,Raze,Reyna,Viper,0.665250
66,Omen,Phoenix,Raze,Reyna,Yoru,0.664000
15,Breach,Omen,Phoenix,Raze,Reyna,0.663250
69,Phoenix,Raze,Reyna,Skye,Viper,0.656667
17,Neon,Omen,Phoenix,Raze,Reyna,0.650750


In [61]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [62]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
1827,Chamber,Jett,Omen,Phoenix,Sage,0.868367
26,Killjoy,Omen,Phoenix,Reyna,Skye,0.865233
5558,Chamber,Omen,Phoenix,Raze,Sage,0.850083
735,Brimstone,Cypher,Fade,Jett,Skye,0.838500
7266,Brimstone,Fade,KAY/O,Killjoy,Yoru,0.838000


In [63]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
8665,Jett,Reyna,Sage,Skye,Viper,0.122833
4255,Fade,Jett,Neon,Reyna,Sova,0.110000
1321,Fade,Jett,Neon,Phoenix,Reyna,0.109500
1924,Neon,Phoenix,Reyna,Sova,Yoru,0.098333
8979,Jett,Neon,Phoenix,Reyna,Sova,0.081500


In [64]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
884,Fade,Jett,Reyna,Sage,Viper,0.919250
8693,Jett,Killjoy,Omen,Reyna,Skye,0.911933
10575,Breach,Chamber,Jett,Reyna,Sage,0.894233
10968,Brimstone,Chamber,Fade,Jett,Sage,0.883233
7217,Brimstone,Killjoy,Raze,Reyna,Sage,0.873417


In [65]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
2863,Astra,Raze,Reyna,Sage,Sova,0.126167
10410,Chamber,KAY/O,Omen,Reyna,Skye,0.106500
7960,Brimstone,Chamber,KAY/O,Reyna,Skye,0.106083
4255,Fade,Jett,Neon,Reyna,Sova,0.105000
10301,Omen,Raze,Reyna,Sage,Sova,0.100617


In [66]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
4573,Chamber,Fade,Jett,Reyna,Viper,0.946657
1619,Chamber,Jett,Sage,Skye,Viper,0.946217
7594,Chamber,Reyna,Sage,Viper,Yoru,0.935417
5770,Chamber,Jett,KAY/O,Reyna,Viper,0.930183
2148,Chamber,Reyna,Sage,Skye,Viper,0.898255


In [67]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
669,Breach,Chamber,Omen,Phoenix,Sage,0.119667
3897,Jett,Neon,Reyna,Skye,Sova,0.115748
4255,Fade,Jett,Neon,Reyna,Sova,0.110533
7239,Brimstone,Jett,KAY/O,Reyna,Sage,0.100500
8425,KAY/O,Neon,Skye,Sova,Viper,0.099333


In [68]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
5923,Breach,Jett,Reyna,Sage,Viper,0.888033
10245,Breach,Brimstone,Chamber,Cypher,Neon,0.856600
5682,Killjoy,Omen,Reyna,Sage,Sova,0.856333
9940,Breach,Chamber,Jett,Reyna,Viper,0.851100
5921,Breach,Brimstone,Jett,Killjoy,Sage,0.839500


In [69]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
4252,Breach,Chamber,Killjoy,Sage,Sova,0.130250
5208,Chamber,Killjoy,Neon,Reyna,Sage,0.124750
7171,Brimstone,KAY/O,Neon,Reyna,Sage,0.122767
528,Chamber,Jett,Omen,Reyna,Sage,0.089083
726,Chamber,Omen,Phoenix,Reyna,Sage,0.083317


In [70]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
9875,Chamber,Omen,Raze,Reyna,Sage,0.912750
5724,Brimstone,Chamber,Jett,Reyna,Sova,0.908350
10641,Killjoy,Neon,Omen,Sage,Skye,0.891333
3335,Breach,Omen,Raze,Reyna,Sage,0.890017
726,Chamber,Omen,Phoenix,Reyna,Sage,0.874067


In [71]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
6647,Brimstone,Chamber,Neon,Reyna,Sage,0.11300
7009,Brimstone,Chamber,Killjoy,Sage,Viper,0.11200
7695,Chamber,KAY/O,Killjoy,Sage,Sova,0.10925
9679,Astra,Brimstone,Chamber,Killjoy,Sage,0.10875
4252,Breach,Chamber,Killjoy,Sage,Sova,0.10175


In [72]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
9875,Chamber,Omen,Raze,Reyna,Sage,0.955752
5770,Chamber,Jett,KAY/O,Reyna,Viper,0.936133
9835,Brimstone,Jett,Killjoy,Reyna,Sage,0.901667
970,Chamber,Jett,Reyna,Sage,Sova,0.873417
869,Brimstone,Chamber,Jett,KAY/O,Reyna,0.857417


In [73]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
11352,Breach,Chamber,KAY/O,Killjoy,Sage,0.121417
4252,Breach,Chamber,Killjoy,Sage,Sova,0.118917
3897,Jett,Neon,Reyna,Skye,Sova,0.118667
7417,Chamber,Jett,KAY/O,Omen,Skye,0.111833
4037,Chamber,KAY/O,Omen,Phoenix,Sage,0.093417


In [74]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
5535,Jett,KAY/O,Reyna,Sage,Viper,0.930271
860,Chamber,Fade,Jett,Sage,Viper,0.892800
4573,Chamber,Fade,Jett,Reyna,Viper,0.883833
6604,Phoenix,Raze,Reyna,Sage,Viper,0.878833
2706,Brimstone,Jett,Raze,Reyna,Sage,0.876500


In [75]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
10999,Brimstone,Fade,Neon,Reyna,Sage,0.131475
8979,Jett,Neon,Phoenix,Reyna,Sova,0.130000
9115,Chamber,Neon,Omen,Sage,Sova,0.109833
8688,Jett,Neon,Phoenix,Raze,Reyna,0.100833
7171,Brimstone,KAY/O,Neon,Reyna,Sage,0.095167


In [76]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model, map_pick, current_agent_list).head()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
7217,Brimstone,Killjoy,Raze,Reyna,Sage,0.828083
10575,Breach,Chamber,Jett,Reyna,Sage,0.827300
9835,Brimstone,Jett,Killjoy,Reyna,Sage,0.825333
9307,Jett,Killjoy,Omen,Sage,Skye,0.810250
10641,Killjoy,Neon,Omen,Sage,Skye,0.808167


In [77]:
predict_best_lineup(model, map_pick, current_agent_list).tail()

,agent1_name,agent2_name,agent3_name,agent4_name,agent5_name,win_prob
3897,Jett,Neon,Reyna,Skye,Sova,0.114167
8423,Jett,KAY/O,Neon,Reyna,Sova,0.111000
4252,Breach,Chamber,Killjoy,Sage,Sova,0.110250
4255,Fade,Jett,Neon,Reyna,Sova,0.105500
8979,Jett,Neon,Phoenix,Reyna,Sova,0.097500
